## Controlling Results

### 1. Ordering results

Whether you return results as nodes or as properties of nodes, you can specify a property value for the ordering. Strings are ordered by their text values. Boolean true comes before false when ordered. Numeric data (including date and datetime properties) are ordered by their numeric value.


```cypher
MATCH (p:Person)
WHERE p.born.year = 1980
RETURN p.name AS name,
p.born AS birthDate
ORDER BY p.born
```

Ordering multiple results:

```cypher
MATCH (p:Person)-[:DIRECTED | ACTED_IN]->(m:Movie)
WHERE p.name = 'Tom Hanks'
OR p.name = 'Keanu Reeves'
RETURN  m.year, m.title
ORDER BY m.year DESC , m.title
```


### 2. Limiting results

Although you can filter queries to reduce the number of results returned, you may also want to limit the number of results returned.

```cypher
MATCH (m:Movie)
WHERE m.released IS NOT NULL
RETURN m.title AS title,
m.released AS releaseDate
ORDER BY m.released DESC LIMIT 100
```

### 3. Skipping some results

In an ordered result set, you may want to control what results are returned. This is useful in an application where pagination is required.

```cypher
MATCH (p:Person)
WHERE p.born.year = 1980
RETURN  p.name as name,
p.born AS birthDate
ORDER BY p.born SKIP 40 LIMIT 10
```

### 4. Eliminating duplicate

```cypher
MATCH (p:Person)-[:DIRECTED | ACTED_IN]->(m:Movie)
WHERE p.name = 'Tom Hanks'
RETURN DISTINCT m.title, m.released
ORDER BY m.title
```

### 5. Changing data returned

You can always change the data that is returned by performing string or numeric operations on the data.

```cypher
MATCH (m:Movie)<-[:ACTED_IN]-(p:Person)
WHERE m.title CONTAINS 'Toy Story' AND
p.died IS NULL
RETURN m.title AS movie,
p.name AS actor,
p.born AS dob,
date().year - p.born.year AS ageThisYear
```

Here is an example where we concatenate string data returned:

```cypher
MATCH (m:Movie)<-[:ACTED_IN]-(p:Person)
WHERE m.title CONTAINS 'Toy Story' AND p.died IS NULL
RETURN 'Movie: ' + m.title AS movie,
p.name AS actor,
p.born AS dob,
date().year - p.born.year AS ageThisYear
```

Conditionally changing data returned

```cypher
MATCH (m:Movie)<-[:ACTED_IN]-(p:Person)
WHERE p.name = 'Henry Fonda'
RETURN m.title AS movie,
CASE
WHEN m.year < 1940 THEN 'oldies'
WHEN 1940 <= m.year < 1950 THEN 'forties'
WHEN 1950 <= m.year < 1960 THEN 'fifties'
WHEN 1960 <= m.year < 1970 THEN 'sixties'
WHEN 1970 <= m.year < 1980 THEN 'seventies'
WHEN 1980 <= m.year < 1990 THEN 'eighties'
WHEN 1990 <= m.year < 2000 THEN 'nineties'
ELSE  'two-thousands'
END
AS timeFrame
```

## Aggregation in Cypher

### 1. Using count() to aggregate data

```cypher
MATCH (a:Person)-[:ACTED_IN]->(m:Movie)<-[:DIRECTED]-(d:Person)
RETURN a.name AS actorName,
d.name AS directorName,
count(*) AS numMovies
ORDER BY numMovies DESC
```

### 2. Returning a list

```cypher
MATCH (p:Person)
RETURN p.name, [p.born, p.died] AS lifeTime
LIMIT 10
```

### 3. Using collect() to create a list

```cypher
MATCH (a:Person)-[:ACTED_IN]->(m:Movie)
RETURN a.name AS actor,
count(*) AS total,
collect(m.title) AS movies
ORDER BY total DESC LIMIT 10
```

Rather than collecting the values of the title properties for movies, you can collect the nodes.

```cypher
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE p.name ='Tom Cruise'
RETURN collect(m) AS tomCruiseMovies
```

### 4. Eliminating duplication in lists

```cypher
MATCH (a:Person)-[:ACTED_IN]->(m:Movie)
WHERE m.year = 1920
RETURN  collect( DISTINCT m.title) AS movies,
collect( a.name) AS actors
```

### 5. Accessing elements of a list

```cypher
MATCH (a:Person)-[:ACTED_IN]->(m:Movie)
RETURN m.title AS movie,
collect(a.name)[0] AS castMember,
size(collect(a.name)) as castSize
```

You can also return a slice of a collection.

```cypher
MATCH (a:Person)-[:ACTED_IN]->(m:Movie)
RETURN m.title AS movie,
collect(a.name)[2..] AS castMember,
size(collect(a.name)) as castSize
```

### 6. Other aggregating functions

- `min()`
- `max()`
- `avg()`
- `stddev()`
- `sum()`

You can either use `count()` to count the number of rows, or alternatively, you can return the size of the collected results. The `size()` function returns the number of elements in a list.

### 7. List comprehension

```cypher
MATCH (m:Movie)
RETURN m.title as movie,
[x IN m.countries WHERE x CONTAINS 'USA' OR x CONTAINS 'Germany']
AS country LIMIT 500
```

### 8. Pattern comprehension

```cypher
MATCH (m:Movie)
WHERE m.year = 2015
RETURN m.title,
[(dir:Person)-[:DIRECTED]->(m) | dir.name] AS directors,
[(actor:Person)-[:ACTED_IN]->(m) | actor.name] AS actors
```

Notice that for pattern comprehension we specify the list with the square braces to include the pattern followed by the pipe character to then specify what value will be placed in the list from the pattern. `[<pattern> | value]`

Here is another example of using pattern comprehension to create a list where we specify a filter for the pattern.

```cypher
MATCH (a:Person {name: 'Tom Hanks'})
RETURN [(a)-->(b:Movie)
WHERE b.title CONTAINS "Toy" | b.title + ": " + b.year]
AS movies
```

### 9. Working with maps

A Cypher map is list of key/value pairs where each element of the list is of the format 'key': value. A node or relationship can have a property that is a map.

```cypher
RETURN {Jan: 31, Feb: 28, Mar: 31, Apr: 30 ,
May: 31, Jun: 30 , Jul: 31, Aug: 31, Sep: 30,
Oct: 31, Nov: 30, Dec: 31}['Feb'] AS daysInFeb
```

Alternatively, you can access a value with the '.' notation:

```cypher
RETURN {Jan: 31, Feb: 28, Mar: 31, Apr: 30 ,
May: 31, Jun: 30 , Jul: 31, Aug: 31, Sep: 30,
Oct: 31, Nov: 30, Dec: 31}.Feb AS daysInFeb
```

And you can return a list of keys of a map as follows:

```cypher
RETURN keys({Jan: 31, Feb: 28, Mar: 31, Apr: 30 ,
May: 31, Jun: 30 ,Jul: 31, Aug: 31, Sep: 30,
Oct: 31, Nov: 30, Dec: 31}) AS months
```

### 10. Map projections

Map projections are when you can use retrieved nodes to create or return some of the information in the nodes.

```cypher
MATCH (m:Movie)
WHERE m.title CONTAINS 'Matrix'
RETURN m { .title, .released } AS movie
```


## Date and Time

### 1. Working with date time data

Cypher has these basic formats for storing date and time data.

```cypher
RETURN date(), datetime(), time()
```

Create a node in the graph containing

```cypher
MERGE (x:Test {id: 1})
SET x.date = date(),
    x.datetime = datetime(),
    x.time = time()
RETURN x
```

### 2. Extracting components of a date or datetime

```cypher
MATCH (x:Test {id: 1})
RETURN x.date.day, x.date.year,
x.datetime.year, x.datetime.hour,
x.datetime.minute
```

### 3. Setting date values

```cypher
MATCH (x:Test {id: 1})
SET x.date1 = date('2022-01-01'),
    x.date2 = date('2022-01-15')
RETURN x
```

### 4. Setting datetime values

```cypher
MATCH (x:Test {id: 1})
SET x.datetime1 = datetime('2022-01-04T10:05:20'),
    x.datetime2 = datetime('2022-04-09T18:33:05')
RETURN x
```

### 5. Working with durations

```cypher
MATCH (x:Test {id: 1})
RETURN duration.between(x.date1,x.date2)
```

We can return the duration in days between two datetime values

```cypher
MATCH (x:Test {id: 1})
RETURN duration.inDays(x.datetime1,x.datetime2).days
```

We can add a duration of 6 months:

```cypher
MATCH (x:Test {id: 1})
RETURN x.date1 + duration({months: 6})
```

### 6. Using APOC to format dates and times

```cypher
MATCH (x:Test {id: 1})
RETURN x.datetime as Datetime,
apoc.temporal.format( x.datetime, 'HH:mm:ss.SSSS')
AS formattedDateTime
```